In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
# 这是什么
from torch.autograd import Variable
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import matplotlib.gridspec as gridspec

In [2]:
# 定义数据
mb_size = 64
Z_dim = 100
X_dim = 784
y_dim = 10
h_dim = 128
c = 0
lr = 1e-3

torch.manual_seed(42)
transform = transforms.Compose([transforms.ToTensor()])
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('../data', train=True, download=True, transform=transform),
    batch_size=mb_size, shuffle=True, drop_last=True)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ../data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ../data\FashionMNIST\raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ../data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ../data\FashionMNIST\raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ../data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ../data\FashionMNIST\raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ../data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ../data\FashionMNIST\raw


In [3]:
def xavier_init(size):
    #
    in_dim = size[0]
    xavier_stddev = 1. / np.sqrt(in_dim / 2.)
    return Variable(torch.randn(*size) * xavier_stddev, requires_grad=True)

In [4]:
# 定义生成器
Wxh = xavier_init(size=[X_dim, h_dim])
bxh = Variable(torch.zeros(h_dim), requires_grad=True)

Whz_mu = xavier_init(size=[h_dim, Z_dim])
bhz_mu = Variable(torch.zeros(Z_dim), requires_grad=True)

Whz_var = xavier_init(size=[h_dim, Z_dim])
bhz_var = Variable(torch.zeros(Z_dim), requires_grad=True)

In [5]:
def Q(x):
    h = F.relu(x @ Wxh + bxh.repeat(x.size(0), 1))
    z_mu = h @ Whz_mu + bhz_mu.repeat(h.size(0), 1)
    z_var = h @ Whz_var + bhz_var.repeat(h.size(0), 1)
    return z_mu, z_var

def sample_z(mu, log_var):
    eps = Variable(torch.randn(mu.size(0), Z_dim))
    return mu + torch.exp(log_var / 2) * eps

Wzh = xavier_init(size=[Z_dim, h_dim])
bzh = Variable(torch.zeros(h_dim), requires_grad=True)

Whx = xavier_init(size=[h_dim, X_dim])
bhx = Variable(torch.zeros(X_dim), requires_grad=True)

def P(z):
    h = F.relu(z @ Wzh + bzh.repeat(z.size(0), 1))
    X = F.sigmoid(h @ Whx + bhx.repeat(h.size(0), 1))
    return X

In [7]:
params = [Wxh, bxh, Whz_mu, bhz_mu, Whz_var, bhz_var,
          Wzh, bzh, Whx, bhx]

solver = optim.Adam(params, lr=lr)

it = 0
while it <= 20000:
    for data, _ in tqdm(train_loader, desc = "Training"):
        it += 1
        X = data.view(-1, 28 * 28)
        X = Variable(X)
        
        # Forward
        z_mu, z_var = Q(X)
        z = sample_z(z_mu, z_var)
        X_sample = P(z)
        
        # Loss
        recon_loss = F.binary_cross_entropy(X_sample, X, size_average=False) / mb_size
        kl_loss = torch.mean(0.5 * torch.sum(torch.exp(z_var) + z_mu**2 - 1. - z_var, 1))
        loss = recon_loss + kl_loss
        
        # Backward
        loss.backward()
        
        # Update
        solver.step()
        
        # 
        solver.zero_grad()
        
        # Print and plot every now and then
        if it % 1000 == 0:
            print('Iter-{}; Loss: {:.4}'.format(it, loss.item()))
            
            samples = P(z).data.numpy()[:16]
            
            fig = plt.figure(figsize=(4, 4))
            gs = gridspec.GridSpec(4, 4)
            gs.update(wspace=0.05, hspace=0.05)
            
            for i, sample in enumerate(samples):
                ax = plt.subplot(gs[i])
                plt.axis('off')
                ax.set_xticklabels([])
                ax.set_yticklabels([])
                ax.set_aspect('equal')
                plt.imshow(sample.reshape(28, 28), cmap='Greys_r')
                
            if not os.path.exists('out/'):
                os.makedirs('out/')
            
            plt.savefig('out/{}.png'.format(str(c).zfill(3)), bbox_inches='tight')
            c += 1
            plt.close(fig)

Training:   6%|▌         | 56/937 [00:00<00:12, 70.09it/s]

Iter-1000; Loss: 263.8


Training:  13%|█▎        | 124/937 [00:01<00:10, 76.47it/s]

Iter-2000; Loss: 246.3


Training:  20%|█▉        | 187/937 [00:02<00:10, 72.36it/s]

...

Training:  27%|██▋       | 254/937 [00:03<00:09, 69.50it/s]

Iter-19000; Loss: 251.5


Training:  34%|███▍      | 322/937 [00:04<00:09, 67.00it/s]

Iter-20000; Loss: 238.8


Training: 100%|██████████| 937/937 [00:15<00:00, 61.21it/s]
